In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-25 21:02:26.559605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 21:02:26.592018: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 21:02:26.753527: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 21:02:26.753639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 21:02:26.792100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)

In [4]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [5]:
regressor = SymbolicRegressor(sampler, n_samples=32)

In [6]:
regressor.compile(loss=SymbolicLoss(rsquared))

In [9]:
regressor.fit(X, y)

ValueError: in user code:

    File "/home/laurent/miniconda3/envs/pysr_env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/laurent/miniconda3/envs/pysr_env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/laurent/miniconda3/envs/pysr_env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/laurent/miniconda3/envs/pysr_env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/laurent/miniconda3/envs/pysr_env/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_g

In [ ]:
regressor.summary()

In [10]:
output = regressor({"X": X, "y": y})

DEBUG - 25-Dec-23 21:04:50 - Fitting 1/32: var_x - cos(log(var_x / var_x)) + var_x * const, object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 2/32: sin(var_x * var_x * var_x + var_x * var_x + const), object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 3/32: var_x + log(log(var_x)) - var_x + var_x - var_x, object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 4/32: cos(cos(var_x)) - var_x - var_x * var_x * var_x, object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 5/32: var_x - exp(cos(exp(var_x / var_x))) - var_x * var_x, object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 6/32: sin(cos(var_x)) - exp(cos(var_x)) - var_x * var_x, object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 7/32: sin(exp(exp(exp(var_x))) + var_x + sin(exp(var_x))), object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 8/32: cos(var_x), object=ExpressionEnsemble
DEBUG - 25-Dec-23 21:04:50 - Fitting 9/32: exp(sin(var_x - var_x) * var_x + var

In [ ]:
Loss = SymbolicLoss(rsquared)

In [ ]:
Loss(tf.convert_to_tensor(y, dtype=tf.float32), output)

# TODO constraint avoid x / x

In [ ]:
from tensorflow.keras.losses import MSE
from scipy.optimize import basinhopping, minimize

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
expression = Expression(tree)

In [ ]:
results = {}
for T in np.logspace(0, 2, 10):
    res = basinhopping(
        lambda constants: MSE(y, expression.eval(X, constants)),
        expression.constants,
        T=T,
    )
    results[T] = res

In [ ]:
[res.x for result in results]

In [ ]:
res = expression.fit(X, y)

In [ ]:
expression.constants = res.x

In [ ]:
go.Figure(
    [go.Scatter(y=expression.eval(X, res.x)), go.Scatter(y=expression.eval(X, [2, 1]))]
)

In [ ]:
model = EvalModel(tree)
model.compile(optimizer=RMSprop(learning_rate=0.05), loss="mse")
model.fit(
    X,
    y,
    batch_size=X.shape[0],
    epochs=1000,
    callbacks=[
        EarlyStopping(monitor="loss", patience=10, start_from_epoch=10, min_delta=0)
    ],
)
model.constants

In [ ]:
def step(tree, X, y, constants, optimizer: Optimizer):
    with tf.GradientTape() as tape:
        y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
        loss = MSE(y, y_pred)
    grads = tape.gradient(loss, constants)
    optimizer.apply_gradients([(grads, constants)])
    return grads, loss


def optimize(
    tree, X: pd.DataFrame, y: pd.Series, constants: tf.Variable, max_steps: int = 100
):
    # optimizer = SGD(learning_rate=0.1)
    optimizer = RMSprop(learning_rate=0.1)

    if isinstance(X, pd.DataFrame):
        X = format_as_dict_of_tensor(X)

    if isinstance(y, pd.Series):
        y = tf.convert_to_tensor(y)

    for n in range(max_steps):
        grads, loss = step(tree, X, y, constants, optimizer)
    return constants


def optimize_constants(tree, X: pd.DataFrame, y: pd.Series, max_steps: int = 100):
    num_constants = walk(tree, count_constant)
    constants = tf.Variable(np.random.randn(num_constants), dtype=tf.float32)
    return optimize(tree, X, y, constants, max_steps)

In [ ]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [ ]:
model.fit(X, y)

In [ ]:
xv, yv = np.meshgrid(np.linspace(-4, 4), np.linspace(-4, 4))

In [ ]:
losses = np.empty_like(xv)
grads = np.empty((50, 50, 2))
for i in range(50):
    for j in range(50):
        constants = tf.Variable([xv[i, j], yv[i, j]])
        with tf.GradientTape() as tape:
            y_pred = walk(tree, tf_eval, X=X, constants={"iter": 0, "value": constants})
            loss = MSE(y, y_pred)
        losses[i, j] = loss.numpy()
        grad = tape.gradient(loss, constants)
        grads[i, j] = grad.numpy()

In [ ]:
go.Figure(
    go.Contour(x=np.linspace(-4, 4), y=np.linspace(-4, 4), z=losses, colorscale="RdBu"),
)

In [ ]:
ff.create_quiver(xv, yv, grads[:, :, 0], grads[:, :, 1], scale=0.05, arrow_scale=0.25)

In [ ]:
losses

In [ ]:
grad

In [ ]:
constants = optimize_constants(tree, X, y)

In [ ]:
constants

In [ ]:
go.Figure(
    [
        go.Scatter(
            x=X.squeeze(),
            y=walk(
                tree, tf_eval, X=X, constants={"iter": 0, "value": constants}
            ).numpy(),
        ),
        go.Scatter(x=X.squeeze(), y=y),
    ]
)